# Cloudinary upload

According to: https://cloudinary.com/documentation/upload_images#uploading_with_a_direct_call_to_the_rest_api

'https://api.cloudinary.com/v1_1/<cloud name>/<resource_type>/upload'


In [ ]:
import os
import re

box_library_root = os.environ['HOME'] + '/Box/Q-bis_Ende_2019/3/31/MediaAssets/Official_Icon_Library/PNG'
assert os.path.isdir(box_library_root), "Is directory"

In [ ]:
from pathlib import Path
result = list(Path(box_library_root).rglob("*.[pP][nN][gG]"))
result[:3]

In [ ]:
filename_id_extractor = re.compile(".*_([0-9]{3,4})_(.+)\.png", re.IGNORECASE)

def icon_id(file):
    base = os.path.basename(file)
    match = filename_id_extractor.match(base)
    if match:
        return match.group(1)
    else:
        return hash(base)

icon_id(result[0])

In [ ]:
import hashlib

def signature(params, secret):
    ''' calculates the sha signatur according to 
        https://cloudinary.com/documentation/upload_images#generating_authentication_signatures
    '''
    sorter = list()
    for entry in params:
        sorter.append(entry + '=' + params[entry] + '&')
    sorter.sort()
    serialized = ''.join(sorter)
    serialized =serialized[:-1] + secret
    print(serialized)
    m = hashlib.sha256()
    m.update(serialized.encode('utf-8'))
    return m.hexdigest()

# eager=w_400,h_300,c_pad|w_260,h_200,c_crop&public_id=sample_image
sample = { 'timestamp': '1315060510', 'eager': 'w_400,h_300,c_pad|w_260,h_200,c_crop', 'public_id': 'sample_image' }
sig = signature(sample, 'abcd') 
sig

In [ ]:
import time
time_stamp = int(time.time())

In [ ]:
import requests
import base64

account='fyayc-test' 
uri='https://api.cloudinary.com/v1_1/' + account + '/image/upload'
api_key='778862129561347'
api_secret='-xJwqK-LkCPVdKZrDosLpesBM5Q'

def upload(file):
    file_name_ext = os.path.basename(file)
    file_name = os.path.splitext(file_name_ext)[0]
    
    file_data = None
    with open(file, 'rb') as binary_file:
        binary_file_data = binary_file.read()
        #base64_encoded_data = base64.b64encode(binary_file_data)
        #base64string = base64_encoded_data.decode('utf-8')
    
    params = {}
    params['timestamp'] = str(time_stamp)
    
    # optinal parameters
    params['public_id'] = file_name
    params['folder'] = 'icons'
    
    sig = signature(params, api_secret)

    # append non signed parameters
    params['api_key'] = api_key
    params['signature'] = sig

    payload = params
 
    files_data = {'file': (file_name, binary_file_data)}
    resp = requests.post(uri + '?timestamp=' + str(time_stamp) + '&api_key=' + api_key + '&signature=' + sig + '&', 
                        #headers = headers, 
                        data = payload,
                        files = files_data)

    return resp

In [ ]:
response = upload(result[0])
vars(response)

In [ ]:
summary = []

for entry in result:
    print('processing ' + str(entry))
    response = upload(entry)
    response.raise_for_status()
    summary.append( { 
        'status': response.status_code, 
        'source': str(entry), 
        'result':response.json(), 
        'id': icon_id(entry)
    })


In [ ]:
summary[:-3]

In [ ]:
import json

with open('upload-summary-' + str(time_stamp) + '.json', 'w') as outfile:
    json.dump(summary, outfile)